In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from sklearn.preprocessing import LabelEncoder
from keras.models import model_from_json


Using TensorFlow backend.
D:\d\softwares\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\d\softwares\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\d\softwares\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\d\softwares\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [2]:
data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]

In [3]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

In [4]:
print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)

4472
16986


In [5]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

In [6]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)
print(X)


[[   0    0    0 ...   54 1065    1]
 [   0    0    0 ... 1571 1420  847]
 [   0    0    0 ...   10  715  528]
 ...
 [   0    0    0 ...   69   61    3]
 [   0    0    0 ... 1109 1582   83]
 [   0    0    0 ...   19    3  877]]


In [7]:
embed_dim = 128
lstm_out = 196
def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    return model


In [8]:
labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)

In [9]:
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)



Epoch 1/5
 - 13s - loss: 0.8326 - accuracy: 0.6408
Epoch 2/5
 - 17s - loss: 0.6859 - accuracy: 0.7084
Epoch 3/5
 - 17s - loss: 0.6245 - accuracy: 0.7377
Epoch 4/5
 - 16s - loss: 0.5740 - accuracy: 0.7644
Epoch 5/5
 - 17s - loss: 0.5307 - accuracy: 0.7826


In [14]:
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
print(score)
print(acc)
print(model.metrics_names)

0.8236752596548312
0.6743119359016418
['loss', 'accuracy']


In [15]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 

Loaded model from disk


In [17]:
twt = ['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump' ]
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)


[[  0   0   0   0   0   0   0   0   0   0   0   7 450   5 148 293  36  31
   36 392   2 351   8 261   7 154 268  24]]


In [18]:
sentiment = loaded_model.predict(twt,batch_size=1,verbose = 2)[0]
print(sentiment)

[0.09778629 0.05151282 0.85070086]


In [19]:
loaded_model.predict(twt,batch_size=1,verbose = 2)

array([[0.09778629, 0.05151282, 0.85070086]], dtype=float32)